In [41]:
import pandas

import numpy
import pygsheets
import datetime
import re

In [57]:
#pandas.io.formats.excel.ExcelFormatter.header_style = None
excel_writer = pandas.ExcelWriter('EuropeGasTracker-Terminals-SummarySheets-'+str(datetime.date.today())+'.xlsx', engine='xlsxwriter')

import Terminals_Current dataset

In [58]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
#gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1BHiidrJnzh96Em3KrqBpCoWaJ3GB5OeCY9dHVaEDEFs') # Europe gas report version
#spreadsheet = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A') # current version
spreadsheet = gc.open_by_key('129b9YeQn7uAIsMgyB5uleT2QZXYxjD_UetnqwVyoRaI')

#spreadsheet[1] "Gas Pipelines" tab is the second index
terms_df_orig = spreadsheet.worksheet('title', 'Terminals').get_as_df(start='A2')

## get other relevant sheets
# owners_df_orig = spreadsheet.worksheet('title', 'Terminal operators/owners (1/3)').get_as_df(start='A2')

## remove empty cells for pipes, owners
# owners_df_orig = owners_df_orig[owners_df_orig['ProjectID']!='']
# owners_df_orig = owners_df_orig[owners_df_orig['Wiki']!='']
# owners_df_orig = owners_df_orig[owners_df_orig.Status!='N/A']

# owners_df_orig.set_index('ProjectID', inplace=True)

In [59]:
terms_df_orig.replace('--', numpy.nan, inplace=True)

#owners_df_orig.replace('--', numpy.nan, inplace=True)
#owners_df_orig.replace('', numpy.nan, inplace=True)

# drop any terminals without a wiki
terms_df_orig = terms_df_orig.loc[terms_df_orig.Wiki!='']

In [60]:
region_df_orig = spreadsheet.worksheet('title','Region dictionary').get_as_df(start='A1')
region_df_egt = region_df_orig.copy()[region_df_orig['EuroGasTracker']=='Yes']
region_df_europe = region_df_orig.copy()[region_df_orig['Region']=='Europe']
region_df_euro_uk = region_df_orig.copy()[(region_df_orig['Region']=='Europe') | 
                                          (region_df_orig['Country'].isin(['United Kingdom','Israel']))]

In [61]:
region_df_touse = region_df_euro_uk.copy()

### create EU-specific dataframes for region, country_ratios, owners_df

In [62]:
# owners_df_touse = owners_df_orig.loc[owners_df_orig['Country'].str.contains(
#                                             '|'.join(region_df_touse['Country'].tolist()))]

terms_df_touse = terms_df_orig.loc[terms_df_orig['Country'].isin(region_df_touse['Country'].tolist())]

### set up info to analyze Owners tab

In [63]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10']

percent_column_list = ['Owner1%', 
                     'Owner2%', 
                     'Owner3%', 
                     'Owner4%', 
                     'Owner5%', 
                     'Owner6%', 
                     'Owner7%', 
                     'Owner8%', 
                     'Owner9%', 
                     'Owner10%']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10']

In [64]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse['Country'].tolist()))
region_list = sorted(set(region_df_touse['Region'].tolist()))

In [65]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

### no. export trains by country/region, project status

### no. export trains by country/region, project status - INCLUDING RE-EXPORTING FACILITIES

In [66]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Export') & (terms_df_touse['Fuel']=='LNG')]
terms_df_reexport_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Import') 
                                                    & (terms_df_touse['Fuel']=='LNG')
                                                    & (terms_df_touse['ReExport']=='Yes')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)
ntrains_reexport_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_reexport_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===region-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby('Region').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_reexport_subset_status = terms_df_reexport_subset.copy()[terms_df_reexport_subset['Status']==status]
    ntrains_reexport_by_country[status] = terms_df_reexport_subset_status.groupby('Country').size()

print('===region-level calculations===')
for status in status_list:
    print(status)
    terms_df_reexport_subset_status = terms_df_reexport_subset.copy()[terms_df_reexport_subset['Status']==status]
    ntrains_reexport_by_region[status] = terms_df_reexport_subset_status.groupby('Region').size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)
ntrains_reexport_by_country = ntrains_reexport_by_country.fillna(0)
ntrains_reexport_by_region = ntrains_reexport_by_region.fillna(0)

# calculate totals
totals_row = ntrains_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_region = ntrains_by_region.append(totals_row)

totals_row = ntrains_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_country = ntrains_by_country.append(totals_row)

totals_row = ntrains_reexport_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_reexport_by_country = ntrains_reexport_by_country.append(totals_row)

totals_row = ntrains_reexport_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_reexport_by_region = ntrains_reexport_by_region.append(totals_row)

# do Prop+Const sum
ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)
ntrains_reexport_by_region['In Development (Proposed + Construction)'] = ntrains_reexport_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_reexport_by_country['In Development (Proposed + Construction)'] = ntrains_reexport_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country.applymap(int).applymap(str)
ntrains_by_region = ntrains_by_region.applymap(int).applymap(str)
ntrains_reexport_by_country = ntrains_reexport_by_country.applymap(int).applymap(str)
ntrains_reexport_by_region = ntrains_reexport_by_region.applymap(int).applymap(str)

# then replace all zeros with empty string
ntrains_reexport_by_country = ntrains_reexport_by_country.applymap('({})'.format)
ntrains_reexport_by_region = ntrains_reexport_by_region.applymap('({})'.format)
ntrains_reexport_by_country = ntrains_reexport_by_country.replace('(0)','')
ntrains_reexport_by_region = ntrains_reexport_by_region.replace('(0)','')

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]
ntrains_reexport_by_country = ntrains_reexport_by_country[excel_status_list]
ntrains_reexport_by_region = ntrains_reexport_by_region[excel_status_list]

ntrains_by_region.index.name = 'Region'
ntrains_by_country.index.name = 'Country'
ntrains_reexport_by_region.index.name = 'Region'
ntrains_reexport_by_country.index.name = 'Country'

ntrains_by_region += ntrains_reexport_by_region
ntrains_by_country += ntrains_reexport_by_country

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG export trains by region')#, index=False)
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG export trains by country')#, index=False)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===region-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===region-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/3068661655.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_region = ntrains_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/3068661655.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_country = ntrains_by_country.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/3068661655.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_reexport_by_country = ntrains_reexport_by_country.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/3068661655.py:56: FutureWarning: The frame.append method is deprecated and will be removed f

### no. import trains by country/region, project status

In [67]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Import') & (terms_df_touse['Fuel']=='LNG')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby('Region').size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)

ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]

ntrains_by_region.index.name = 'Region'
ntrains_by_country.index.name = 'Country'

totals_row = ntrains_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_region = ntrains_by_region.append(totals_row)

totals_row = ntrains_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_country = ntrains_by_country.append(totals_row)

ntrains_by_country = ntrains_by_country.applymap(int).applymap(str)
ntrains_by_region = ntrains_by_region.applymap(int).applymap(str)

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG import trains by region')
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG import trains by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/2444430388.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_region = ntrains_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/2444430388.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_country = ntrains_by_country.append(totals_row)


### capacity (mtpa) of import trains by country/region, project status

In [68]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Import') & (terms_df_touse['Fuel']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby('Region')['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.name = 'Region'
cap_by_country.index.name = 'Country'

totals_row = cap_by_region.sum(axis=0)
totals_row.name = 'Total'
cap_by_region = cap_by_region.append(totals_row)

totals_row = cap_by_country.sum(axis=0)
totals_row.name = 'Total'
cap_by_country = cap_by_country.append(totals_row)

cap_by_region.to_excel(excel_writer, 'LNG import capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG import capacity by country')

===country-level calculations===

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/3933036309.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_region = cap_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/3933036309.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_country = cap_by_country.append(totals_row)



Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


### capacity (mtpa) of export trains by country/region, project status

In [69]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Export') & (terms_df_touse['Fuel']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby('Region')['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.name = 'Region'
cap_by_country.index.name = 'Country'

totals_row = cap_by_region.sum(axis=0)
totals_row.name = 'Total'
cap_by_region = cap_by_region.append(totals_row)

totals_row = cap_by_country.sum(axis=0)
totals_row.name = 'Total'
cap_by_country = cap_by_country.append(totals_row)

cap_by_region.to_excel(excel_writer, 'LNG export capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG export capacity by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/4102405129.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_region = cap_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_7559/4102405129.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_country = cap_by_country.append(totals_row)


## owner analysis

In [71]:
owner_parent_calculations_df = pandas.DataFrame()

for idx,row in terms_df_orig.iterrows():
    parent_string = row.Parent
    #print(parent_string)
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ComboID':row.ComboID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'CapacityInMtpa':row.CapacityInMtpa}])])

owner_parent_calculations_df['CapacityInMtpaOwnership'] = owner_parent_calculations_df['FractionOwnership']*owner_parent_calculations_df['CapacityInMtpa']

,Parent,ComboID,FractionOwnership,CapacityInMtpa,CapacityInMtpaOwnership
0,Energy World,T020501,1.000000,2.00,2.00
0,Energy World,T020502,1.000000,3.00,3.00
0,State of Alaska,T020601,1.000000,6.70,6.70
0,State of Alaska,T020602,1.000000,6.70,6.70
0,State of Alaska,T020603,1.000000,6.70,6.70
...,...,...,...,...,...
0,Vopak,T108400,0.333333,5.88,1.96
0,PT Petrogas Jatim Utama,T108500,0.500000,NaN,NaN
0,Woodside Energy,T108500,0.500000,NaN,NaN
0,Unknown,T108600,1.000000,NaN,NaN


### ntrains export by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Import/Export']=='Export']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_ntrains_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        ntrain_fractions = numpy.array(row_fractions)

        for owner_idx in range(n_owners):
            owners_ntrains_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_fractions[owner_idx]

owners_ntrains_by_status_df.index.name = 'Parent Company'
owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

totals_row = owners_ntrains_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)

#owners_ntrains_by_status_df.to_excel('EuroGas'+'LNGExportTrainsByOwnerAndStatus.xlsx')
owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG export trains by owner')

### ntrains import by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Import/Export']=='Import']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_ntrains_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        ntrain_fractions = numpy.array(row_fractions)

        for owner_idx in range(n_owners):
            owners_ntrains_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_fractions[owner_idx]

owners_ntrains_by_status_df.index.name = 'Parent Company'
owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

totals_row = owners_ntrains_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)

#owners_ntrains_by_status_df.to_excel('EuroGas'+'LNGImportTrainsByOwnerAndStatus.xlsx')
owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG import trains by owner')

### capacity import by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Import/Export']=='Import']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_capacity_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        train_capacity = row['CapacityInMtpa']
        ntrain_capacity_fractions = numpy.array(row_fractions)*train_capacity

        for owner_idx in range(n_owners):
            owners_capacity_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_capacity_fractions[owner_idx]

owners_capacity_by_status_df.index.name = 'Parent Company'
owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

totals_row = owners_capacity_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)

#owners_capacity_by_status_df.to_excel('EuroGas'+'LNGImportTrainCapacityByOwnerAndStatus.xlsx', na_rep='--')
owners_capacity_by_status_df.to_excel(excel_writer, 'LNG import capacity by owner')

### capacity export by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Import/Export']=='Export']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_capacity_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        train_capacity = row['CapacityInMtpa']
        ntrain_capacity_fractions = numpy.array(row_fractions)*train_capacity

        for owner_idx in range(n_owners):
            owners_capacity_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_capacity_fractions[owner_idx]

owners_capacity_by_status_df.index.name = 'Parent Company'
owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

totals_row = owners_capacity_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)

#owners_capacity_by_status_df.to_excel('EuroGas'+'LNGExportTrainCapacityByOwnerAndStatus.xlsx', na_rep='--')
owners_capacity_by_status_df.to_excel(excel_writer, 'LNG export capacity by owner')

### no. of operating terminals by start year, Import/Export type (1980–2021)

In [ ]:
terms_started_eu = terms_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

year_indices = list(range(1980,2022))

terms_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export trains'])

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Fuel']=='LNG') & (terms_started_eu['Import/Export']=='Import')]
terms_by_start_year_df['Import terminals'] = terms_started_eu.groupby('StartYearEarliest').size()

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Fuel']=='LNG') & (terms_started_eu['Import/Export']=='Export')]
terms_by_start_year_df['Export trains'] = terms_started_eu.groupby('StartYearEarliest').size()

terms_by_start_year_df.index.name = 'Start year'
terms_by_start_year_df.replace(numpy.nan,0,inplace=True)

totals_row = terms_by_start_year_df.sum(axis=0)
totals_row.name = 'Total'
terms_by_start_year_df = terms_by_start_year_df.append(totals_row)

terms_by_start_year_df.to_excel(excel_writer, 'LNG terminals by start year')

### capacity of operating terminals by start year, Import/Export type (1980–2021)

In [ ]:
terms_started_eu = terms_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

year_indices = list(range(1980,2022))

capacity_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export trains'])

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Fuel']=='LNG') & (terms_started_eu['Import/Export']=='Import')]
capacity_by_start_year_df['Import terminals'] = terms_started_eu.groupby('StartYearEarliest')['CapacityInMtpa'].sum()

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Fuel']=='LNG') & (terms_started_eu['Import/Export']=='Export')]
capacity_by_start_year_df['Export trains'] = terms_started_eu.groupby('StartYearEarliest')['CapacityInMtpa'].sum()

capacity_by_start_year_df.index.name = 'Start year'
capacity_by_start_year_df.replace(numpy.nan,0,inplace=True)

totals_row = capacity_by_start_year_df.sum(axis=0)
totals_row.name = 'Total'
capacity_by_start_year_df = capacity_by_start_year_df.append(totals_row)

capacity_by_start_year_df.to_excel(excel_writer, 'LNG capacity by start year')

## save excel file

In [ ]:
excel_writer.save()